----
<img src="../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Define and Open a session

Depending on the Credentials and Refinitiv Products/Services you have access to, you need define & open either a Platform Session or Desktop Session: 

 - **Platform Session:** Used to connect to the Refinitiv Data Platform (RDP) directly or via a local (deployed) Real-time Distribution System(RTDS). The PlatformSession requires RDP credentials (machine account) or connection details of your local platform (IP of the local server and DACS user name).
  - When the PlatformSession is used with RDP credentials only, it retrieves both streaming data and non-streaming data from the Refinitiv Data Platform.
  - When the PlatformSession is used with connection details of your local deployed platform, it retrieves streaming data only from your local platform.  
   
   
 - **Desktop Session:** This type of session is used to connect RDP either via Eikon or via the Refinitiv Workspace. Eikon or the Refinitiv Workspace must be run **on the same PC** as your Python code.
 
You will also require an App Key that uniquely identifies your application. Please refer to the Refinitiv Data Libraries Quick Start guide's **Access Credentials** section for instructions on creating an App Key.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as detailed in the next section below.

Typically you may place this file in the same folder as your Notebook or Python script.
OR, you can place the file in an alternative folder and specify the folder path using the _RD_LIB_CONFIG_PATH_ environment variable. 

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [18]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "/Users/lydiamarsden/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents/programming/keys-and-config"

## Import the library

In [10]:
import refinitiv.data as rd

## Using the 'refinitiv-data.config.json' file
Using the Refinitiv data library config file allows you to specify various parameters such as:
* Default session
* Credentials
* Connectivity parameters  
* Logging related:   
  - Enable or Disable console/file logging
  - Log Level
  - Filename

### Easiest way to Create & Open a Session

Using the above json config file, where possible, provides the simplest approach to creating and opening sessions.  
For example, assuming you have 
* a '*refinitiv-data.config.json*' in the ***Configuration*** directory 
* **populated the file with your credentials**
* and specified a 'default' session in the config file    

you can create and open the default session as follows:

In [5]:
rd.open_session()

RDError: Error code 0 | Error happened during config file /Users/lydiamarsden/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents/programming/keys-and-config/refinitiv-data.config.json/refinitiv-data.config.json read

The above will look for an entry such as:  
>    *"sessions": {  
>        "default": "desktop.workspace",  
>        ....  
>    }*  

and then use the parameters defined in the config section - in this example *"desktop.workspace"*, to create and open a desktop session.  
Please refer to the supplied example **'refinitiv-data.config.json'** file for the full config example.

### Open a specific configured session
If you have multiple sessions configurations specified in the json file you can open a specific one by name e.g.:

In [17]:
rd.open_session(name='desktop.workspace')

<refinitiv.data.session.Definition object at 0x11cd2f050 {name='workspace'}>

The above would open a session based on the parameters specified in the '*"desktop.workspace"* section of the json file.

#### Open a session defined in a custom configuration file 
As well as the default config filename of **'refinitiv-data.config.json'**, you can also use a custom file to specify the config.   
The following call opens the default session as defined in a custom configuration file - with the filename passed as a parameter.

In [ ]:
rd.open_session(config_name="../Configuration/refinitiv-data.custom.config.json")

#### Close the default session 

To close the default session you can call:

In [13]:
rd.close_session()

<br>      
Alternatively, if you do not wish to use a refinitiv-data config file, you can define sessions by specifying the parameters and session type explicitly - as shown below.

## Define the session using your own credentials store
Sessions can be created by explicitly passing in your credentials/connectivity parameters.
For example, the following calls use credentials/connectivity parameters defined in the cell below.
Please edit the following cell to set your credentials and execute, before continuing.

In [ ]:
APP_KEY                     = 'YOUR_APP_KEY'
RDP_LOGIN                   = 'YOUR_REFINITIV_DATA_PLATFORM_LOGIN'
RDP_PASSWORD                = 'YOUR_REFINITIV_DATA_PLATFORM_PASSWORD'
REALTIME_DISTRIBUTION_SERVICE_HOST      = 'THE_HOST:PORT_OF_YOUR_RTDS_HOST'  
REALTIME_DISTRIBUTION_SERVICE_USER_NAME = 'YOUR_USER_NAME_ON_YOUR_RTDS_HOST' 

   
However, rather than hardcoding parameters in your code, you may prefer to keep your credentials in a .env file (or some other external store) - to avoid accidentally exposing your credentials when sharing code.  
A .env file with blank entries has been included in this ***Quick Start*** folder - for your reference.


In [ ]:
# OR 
# Load credentials from a .env file (+ override any OS system env vars of the same name)
from dotenv import load_dotenv
# change the path to the location of your .env file
load_dotenv(dotenv_path='c:/Refinitiv/.env',override=True)
APP_KEY                     = os.getenv('APP_KEY')       # As created using the AppKey generator
RDP_LOGIN                   = os.getenv('RDP_LOGIN')     # e.g. 'GE-A-01234567-8-9101'
RDP_PASSWORD                = os.getenv('RDP_PASSWORD')  # the long password set via the link in the Machine Account Welcome Email
REALTIME_DISTRIBUTION_SERVICE_HOST = os.getenv('DEPLOYED_PLATFORM_HOST')      # ADS hostname:PORT or IP:PORT e.g. 'myADS:15000'
REALTIME_DISTRIBUTION_SERVICE_USER_NAME = os.getenv('DEPLOYED_PLATFORM_USER_NAME') # Your DACS User name

#### Desktop Session

Create and open a Desktop session to Refintiv Eikon 4 or Refinitiv Workspace application - running locally (using the **APP_KEY** defined in the above notebook).

In [21]:
session = rd.session.desktop.Definition(app_key = APP_KEY).get_session()

In [92]:
import pandas as pd

names = pd.read_csv('/Users/lydiamarsden/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents/programming/Mini projects/Trase_finance_supply_chains/outputs/orbis_company_list_amazon_soy.csv')
names = pd.DataFrame(names)

results = pd.DataFrame(columns=['BusinessEntity', 'DocumentTitle', 'PI'])

for name in names['exporter']:
    result = rd.discovery.search(
        view = rd.discovery.Views.ORGANISATIONS,
        query = name
    )
    result['Name'] = name
    results = pd.concat(objs=[results, result])

#results = results.to_string()
results.to_csv("/Users/lydiamarsden/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents/programming/Mini projects/eikon_api/rdp_name_search_test.csv")




#### OR Platform Session
Create and open a Platform session to connect directly to the Refinitiv Data Platform. 

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
).get_session()

#### OR Local(Deployed) Session
Create and open a Platform session to connect to a local (deployed) real-time distribution system(RTDS). 

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    deployed_platform_host = REALTIME_DISTRIBUTION_SERVICE_HOST,
    deployed_platform_username = REALTIME_DISTRIBUTION_SERVICE_USER_NAME
).get_session()

#### OR Local + Platform

Create and open a Platform session to both the Data Platform and a deployed real-time distribution system (RTDS).  
With this configuration, streaming data is retrieved from the local platform while non-streaming data is retrieved from the Refinitiv Data Platform.  
This scenario could apply where you want to consume both internally published streaming data and non-streaming data from Refinitiv

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    ),
    deployed_platform_host = REALTIME_DISTRIBUTION_SERVICE_HOST,
    deployed_platform_username = REALTIME_DISTRIBUTION_SERVICE_USER_NAME
).get_session()

## Then open it

In [ ]:
session.open()

## Diagnose any Session issues
If you cannot open a session, to generate addtional logging, enable DEBUG logging and call session.open() again

In [ ]:
import logging.config
session.set_log_level(logging.DEBUG)
session.open()
# To set logging back to default value
session.set_log_level(logging.WARNING)

## Close the session

In [ ]:
session.close()

## Session Events and State change notification
If you want to be notified of any Session events or change in State you can optionally specify State and Event callbacks

In [ ]:
# Callback to handle session state changes
def on_state_cb(state_code, state_msg, session):
    print(f"Session State:{state_code} | {state_msg}")
# Callback to handle session events
def on_event_cb(event_code, event_msg, session):
    print(f"Session event:{event_code} | {event_msg}")

# Using PlatformSession - can be applied to DesktopSession as well
session = rd.session.platform.Definition(
    app_key=APP_KEY,
    grant=rd.session.platform.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
).get_session()

# Set your callbacks for handing the session events
session.on_event(on_event_cb)
session.on_state(on_state_cb)

# Open the Session
session.open()

#
# Run your application code
#


In [ ]:
# close the session when done
session.close()